In [1]:
import numpy as np
from scipy.integrate import solve_ivp
from scipy.optimize import root, fixed_point
import matplotlib.pyplot as plt

In [2]:
TFtot = 2000
kon = 0.0016399 
koff_invert = 0.34393 / 4
koff_amp = 0.34393 * 2
kbasal_invert = 0.02612 
kbasal_amp = 0.02612
kmax = 13.588
Kd = 956.75
n = 4.203
kdegR = 0.042116
ktrans = 1.4514
kdegP = 0.007

p_amp =  kon, TFtot, koff_amp, kbasal_amp, kmax, n, Kd, kdegR, ktrans, kdegP
p_invert = kon, TFtot, koff_invert, kbasal_invert, kmax, n, Kd, kdegR, ktrans, kdegP

Kd_scaled = Kd / TFtot
kmax_scaled_amp = kmax * kdegR / (kbasal_amp + kmax)
kbasal_scaled_amp = kbasal_amp * kdegR / (kbasal_amp + kmax)
k_alpha_amp = kon * 1000 / koff_amp
k_beta_amp = koff_amp 

p_scaled_amp = k_alpha_amp, k_beta_amp, kbasal_scaled_amp, kmax_scaled_amp, n, Kd_scaled, kdegR


Kd_scaled = Kd / TFtot
kmax_scaled_invert = kmax * kdegR / (kbasal_invert + kmax)
kbasal_scaled_invert = kbasal_invert * kdegR / (kbasal_invert + kmax)
k_alpha_invert = kon * 1000 / koff_invert
k_beta_invert = koff_invert

p_scaled_invert = k_alpha_invert, k_beta_invert, kbasal_scaled_invert, kmax_scaled_invert, n, Kd_scaled, kdegR


In [3]:
def khammash_ode_scaled(t,x,p_scaled,I):
    if callable(I):
        I = I(t)

    # Parameters
    k_alpha, k_beta, kbasal_scaled, kmax_scaled, n, Kd_scaled, kdegR = p_scaled

    # Variables
    TFon, mRNA = x
    TFon = np.max([0,TFon])

    # k_alpha = kon / koff
    # k_beta = koff

    dTFondt = (I * k_alpha * (1 - TFon) - TFon) * k_beta
    # dTFondt = I * kon * (1 - TFon) - koff * TFon
    dmRNAdt = kbasal_scaled + kmax_scaled * (TFon ** n) / ((Kd_scaled ** n) + (TFon ** n)) - kdegR * mRNA

    return [dTFondt, dmRNAdt]


def khammash_ode(t,x,p,I):
    if callable(I):
        I = I(t)

    # Parameters
    kon, TFtot, koff, kbasal, kmax, n, Kd, kdegR, ktrans, kdegP = p

    # Variables
    TFon, mRNA = x
    TFon = np.max([0,TFon])
    mRNA = mRNA * (kbasal + kmax) / kdegR

    dTFondt = I * kon * (TFtot - (TFon  * TFtot)) - koff * (TFon  * TFtot)
    dmRNAdt = kbasal + kmax * ((TFon  * TFtot) ** n) / ((Kd ** n) + ((TFon  * TFtot) ** n)) - kdegR * mRNA

    dTFondt = dTFondt / TFtot
    dmRNAdt = dmRNAdt * kdegR / (kbasal + kmax)

    return [dTFondt, dmRNAdt]

def fsolve_func(x,period,I,max_step):
    tspan = [0,period]
    sol = solve_ivp(khammash_ode, t_span=tspan, y0=x, args=(p_scaled,I),
                        rtol=1e-9, atol=1e-12, 
                        # t_eval=np.linspace(tspan[0], tspan[1], 1000000), 
                        method='BDF',
                        first_step = 0.0001,
                        max_step=max_step,
                        )
    # return (sol.y[:,-1] - x[:])
    return sol.y[:,-1]
    # return (sol.y[:,-1] - x) * [1/10,10]

def fsolve_func_scaled(x,period,I,max_step, p_scaled):
    tspan = [0,period]
    sol = solve_ivp(khammash_ode_scaled, t_span=tspan, y0=x, args=(p_scaled,I),
                        rtol=1e-9, atol=1e-12, 
                        # t_eval=np.linspace(tspan[0], tspan[1], 1000000), 
                        method='BDF',
                        first_step = 0.0001,
                        max_step=max_step,
                        )
    return (sol.y[:,-1] - x[:])
    # return sol.y[:,-1]
    # return (sol.y[:,-1] - x) * [1/10,10]

def I_fun_base(t, u, period, Imax):
    remainder = t - np.floor(t/period) * period
    I_out = np.zeros(np.array(t).size)
    I_out[remainder < u * period] = Imax
    return I_out

def steady_mRNA_solve(u, period, Imax, p_scaled, x_guess=None):

    I = lambda t: I_fun_base(t, u, period, Imax)

    if x_guess is None:
        x_guess = [u,u]

    if u > 0:
        max_step = u*period
    else:
        max_step = period / 10

    # try:
    #     sol = fixed_point(fsolve_func_scaled, x_guess, args=(period,I,max_step), xtol=1e-9, maxiter=5000)
    # except:
    #     print('first one failed')
    #     x_guess = [u,u]
    #     sol = fixed_point(fsolve_func_scaled, x_guess, args=(period,I,max_step/10), xtol=1e-9, maxiter=5000)
    # steady = sol


    sol = root(fsolve_func_scaled, x_guess, args=(period,I,max_step, p_scaled), tol=1e-9)
    if not sol.success:
        x_guess = [u,u]
        sol = root(fsolve_func_scaled, x_guess, args=(period,I,max_step, p_scaled), tol=1e-9)

    steady = np.maximum(sol.x,[0,0])
    tspan = [0,period]
    sol = solve_ivp(khammash_ode_scaled, t_span=tspan, y0=steady, args=(p_scaled,I),
                        rtol=1e-9, atol=1e-12, 
                        # t_eval=np.linspace(tspan[0], tspan[1], 1000000), 
                        method='BDF',
                        first_step = 0.0001,
                        max_step=max_step,
                        )
    steady = np.trapz(sol.y, x=sol.t) / period

    return steady#, sol.success, sol.message, sol.fun

def opto_growth(t,x,p, od_sp=None):

    # Parameters
    kdegR, ktrans, kdegP, kdegPPDC, mu_max_invert, mu_max_amp, Ks, Y, K_Rep, K_Amp, n_Rep, n_Amp = p

    # x_guess = x
    global setpoint_amp, setpoint_invert
    # if setpoint is None:
    #     out, ier = steady_mRNA_solve(u, period, Imax, x_guess)
    # else:
    #     out = setpoint
    # if ier != 1:
    #     out, ier = steady_mRNA_solve(u, period, Imax, x_guess=None)

    tau_mRNA = 1 / kdegR

    # Variables
    mRNA_invert, mRNA_amp, Rep, Amp, Enz_invert, Enz_amp, S, X_invert, X_amp = x
    # mRNA_blind = 1

    

    mu_x_invert = mu_max_invert * Enz_invert * S / (Ks + S)
    mu_x_amp = mu_max_amp * Enz_amp * S / (Ks + S)

    dmRNA_invert_dt = (setpoint_invert[1] - mRNA_invert) / tau_mRNA
    dmRNA_amp_dt = (setpoint_amp[1] - mRNA_amp) / tau_mRNA

    # Enz_opto = Enz_opto / ((mu_max_opto + kdegP) / ktrans)
    ktrans_scaled_invert = ktrans  * ((mu_max_invert + kdegP) / ktrans)
    ktrans_scaled_PDC_invert = ktrans  * ((mu_max_invert + kdegPPDC) / ktrans)
    
    dRepdt = ktrans_scaled_invert * mRNA_invert * S / (Ks + S) - (kdegP + mu_x_invert) * Rep

    if Rep<0:
        dEnz_invert_dt = ktrans_scaled_PDC_invert * S / (Ks + S) - (kdegPPDC + mu_x_invert) * Enz_invert
    else:
        dEnz_invert_dt = ktrans_scaled_PDC_invert * (K_Rep ** n_Rep / (K_Rep ** n_Rep + Rep ** n_Rep)) * S / (Ks + S) - (kdegPPDC + mu_x_invert) * Enz_invert
    # dEnz_optodt = dEnz_optodt * ((mu_max_opto + kdegP) / ktrans)

    # Enz_blind = Enz_blind / ((mu_max_blind + kdegP) / ktrans)
    ktrans_scaled_amp = ktrans  * ((mu_max_amp + kdegP) / ktrans)
    ktrans_scaled_PDC_amp = ktrans  * ((mu_max_amp + kdegPPDC) / ktrans)
    
    dAmpdt = ktrans_scaled_amp * mRNA_amp * S / (Ks + S) - (kdegP + mu_x_amp) * Amp

    if Amp < 0:
        dEnz_amp_optodt =  - (kdegPPDC + mu_x_amp) * Enz_amp
    else:
        dEnz_amp_optodt = ktrans_scaled_PDC_amp * (Amp ** n_Amp / (K_Amp ** n_Amp + Amp ** n_Amp)) * S / (Ks + S) - (kdegPPDC + mu_x_amp) * Enz_amp
    # dEnz_blind_optodt = dEnz_blind_optodt * ((mu_max_blind + kdegP) / ktrans)


    S0 = 20
    if od_sp is None:
        d = 0
    else:
        d = (mu_x_invert * X_invert + mu_x_amp * X_amp) / (X_invert + X_amp)
        if (X_invert + X_amp) < od_sp:
            d = 0
        elif (X_invert + X_amp) > od_sp:
            d = d #* (X_invert + X_amp)/od_sp
    
    # if setpoint[1] > 0.5:
    #     print(d)
    dSdt = - (mu_x_invert / Y) * X_invert - (mu_x_amp / Y) * X_amp + d * (S0 - S)
    dX_invert_dt = mu_x_invert * X_invert - d * X_invert
    dX_amp_dt = mu_x_amp * X_amp - d * X_amp

    # print(x)
    # print([dmRNA_invert_dt, dmRNA_amp_dt, dRepdt, dAmpdt, dEnz_invert_dt, dEnz_amp_optodt, dSdt, dX_invert_dt, dX_amp_dt])
    return [dmRNA_invert_dt, dmRNA_amp_dt, dRepdt, dAmpdt, dEnz_invert_dt, dEnz_amp_optodt, dSdt, dX_invert_dt, dX_amp_dt]


In [4]:
kdegR = 0.042116
ktrans = 1.4514
kdegP = 0.00
K_Rep = 0.01
K_Amp = 0.99

mu_max_invert = 0.0233 / 20
mu_max_amp = 0.014 / 20

# mu_max = mu_max_opto
Ks = 1
Y = 0.5

In [5]:
# u_arr = np.flip(np.concatenate((np.array([0]),10 ** np.linspace(-3,0,20))))

In [6]:
from tqdm import tqdm

u_arr = np.flip(np.concatenate((np.array([0]),10 ** np.linspace(-3,0,100))))

# u_arr = np.flip(10 ** np.linspace(-1,0,100))
Enz_arr = np.zeros(u_arr.size)
mRNA_arr_amp = np.zeros(u_arr.size)
mRNA_arr_invert = np.zeros(u_arr.size)

setpoint_amp = None
setpoint_invert = None

for i, u in tqdm(enumerate(u_arr), total=u_arr.size):
    # print(u)
    period = 100 / 60
    Imax = 1
    setpoint_amp = steady_mRNA_solve(u, period, Imax, p_scaled_amp, x_guess=setpoint_amp)
    # print(setpoint)
    mRNAss = setpoint_amp[1]
    mRNA_arr_amp[i] = mRNAss

    setpoint_invert = steady_mRNA_solve(u, period, Imax, p_scaled_invert, x_guess=setpoint_invert)

    mRNAss = setpoint_invert[1]
    mRNA_arr_invert[i] = mRNAss

    


100%|██████████| 101/101 [07:58<00:00,  4.74s/it]


In [2]:
from scipy.optimize import fsolve

# Inverted Circuit
# 
# u_arr = np.flip(np.concatenate((np.array([0]),10 ** np.linspace(-3,0,20))))

kdegR = 0.042116
ktrans = 1.4514
kdegP = np.log(2) / (6 * 60)
kdegPPDC = np.log(2) / (12 * 60)
K_Rep = 0.3
n_Rep = 1.5
mu_max_invert = 0.47 / 60

K_Amp = 0.2
mu_max_amp = 0.47 / 60
n_Amp = 1.1

optogrowth_p = kdegR, ktrans, kdegP, kdegPPDC, mu_max_invert, mu_max_amp, Ks, Y, K_Rep, K_Amp, n_Rep, n_Amp


od_sp = 0.01

S = 20 - 2 / Y

steady_state_arr_invert = np.zeros((u_arr.size, 9))
steady_state_arr_amp = np.zeros((u_arr.size, 9))

x_init = [mRNA_arr_amp[0], mRNA_arr_amp[0], mRNA_arr_amp[0], mRNA_arr_amp[0], mRNA_arr_amp[0], mRNA_arr_amp[0], 20, 0, 0.0]
x_init_inv = [mRNA_arr_invert[0], mRNA_arr_invert[0], mRNA_arr_invert[0], mRNA_arr_invert[0], mRNA_arr_invert[0], mRNA_arr_invert[0], 20, 0, 0.0]

for index, u in tqdm(enumerate(u_arr), total=u_arr.size):
    # print(u)
    # period = 100 / 60
    # Imax = 1
    # setpoint = steady_mRNA_solve(u, period, Imax, x_guess=setpoint)
    # # print(setpoint)
    # mRNAss = setpoint[1]
    # mRNA_arr[i] = mRNAss

    setpoint_invert = np.array([0, mRNA_arr_invert[index]])
    setpoint_amp = np.array([0, mRNA_arr_amp[index]])
    # mRNA_invert, mRNA_amp, Rep, Amp, Enz_invert, Enz_amp, S, X_invert, X_amp = x


    x_init_inv[-2] = 0.1
    x_init_inv[-1] = 0.
    tspan = np.array([0, 500*60])
    optogrowth_p = kdegR, ktrans, kdegP, kdegPPDC, mu_max_invert, 0, Ks, Y, K_Rep, K_Amp, n_Rep, n_Amp

    sol = solve_ivp(opto_growth, t_span=tspan, y0=x_init_inv, args=(optogrowth_p,od_sp),
                        rtol=1e-6, atol=1e-9, 
                        method='BDF',
                        first_step = 0.0001,
                        )
#     print('solveIvp', sol.message)
    x_init_inv = sol.y[:,-1]

    # plt.figure()
    # for i in range(9):
    #     plt.plot(sol.t, sol.y[i,:]/max(sol.y[i,:]), label=i)
    # plt.plot(sol.t / 60, sol.y[4,:], label='Enz_invert')
    # plt.plot(sol.t / 60, sol.y[7,:], label='X_invert')
    mask = np.array([1, 1, 1, 1, 1, 1, 0, 0, 0])
    steady_state, infodict, ier, mesg = fsolve(lambda x: opto_growth(0, x, optogrowth_p)*mask, x_init_inv, xtol=1e-6, maxfev=100000, full_output=True, factor=0.1)
    # print(mesg)
    # print('invert' , mesg)
    steady_state_arr_invert[index,:] = steady_state

    optogrowth_p = kdegR, ktrans, kdegP, kdegPPDC, 0, mu_max_amp, Ks, Y, K_Rep, K_Amp, n_Rep, n_Amp


    tspan = np.array([0, 500*60])
    x_init[-2] = 0.
    x_init[-1] = 0.1
    sol = solve_ivp(opto_growth, t_span=tspan, y0=x_init, args=(optogrowth_p,od_sp),
                        rtol=1e-6, atol=1e-9, 
                        method='BDF',
                        first_step = 0.0001,
                        )
#     print('solveIvp', sol.message)
    x_init = sol.y[:,-1]
#     plt.figure()
#     plt.plot(sol.t / 60, sol.y[5,:], label='Enz_amp')


    steady_state, infodict, ier, mesg = fsolve(lambda x: opto_growth(0, x, optogrowth_p)*mask, x_init, xtol=1e-9, maxfev=100000, full_output=True, factor=0.1)
#     print('amp', mesg)
    steady_state_arr_amp[index,:] = steady_state

    # print(steady_state)
#     if index == 0:
#         assert False

Enz_invert_arr = steady_state_arr_invert[:, 4]
Enz_amp_arr = steady_state_arr_amp[:, 5]
S0 = 20
mu_arr_invert = mu_max_invert * Enz_invert_arr * S0 / (Ks + S0) * 60
mu_arr_amp = mu_max_amp * Enz_amp_arr * S0 / (Ks + S0) * 60

# mRNA_invert, mRNA_amp, Rep, Amp, Enz_invert, Enz_amp, S, X_invert, X_amp = x


# steady_state, infodict, ier, mesg = fsolve(lambda x: opto_growth(0, x, optogrowth_p, od_sp), x_init, xtol=1e-12, maxfev=10000, full_output=True)

## Real Data
mu_SGy104_1 = np.array([0.429016493, 0.419454997, 0.425278485, 0.468815929, 0.214066814, 0.202848859, 0.077447387, 0.067285511, 0.487083379, 0.487241995,
0.436233624, 0.416734105, 0.433773399, 0.4552301, 0.406627518, 0.322757789])
mu_SGy104_1_lights = np.array([0, 0, 0.01, 0.01, 0.1, 0.1, 1, 1, 0.03, 0.03, 0.01, 0.01, 0.06, 0.06, 0.08, 0.08])

mu_SGy128_4 = np.array([0.07654071, 0.26858448, 0.28688688, 0.19000113, 0.3834206 ,
       0.42503585, 0.25622547, 0.26932193, 0.30298274, 0.29615459,
       0.28158447, 0.26857221, 0.34449419, 0.38737338, 0.412247  ,
       0.43290314])
mu_SGy128_4_lags = np.array([ 0.        , 39.88279932, 19.98011315, 14.4137813 ,  0.        ,
        2.18260617,  0.        ,  1.21280087, 10.40812555,  7.77763666,
       21.22069601, 19.70771971,  1.68719277,  3.11811039,  2.52691981,
        2.30179095])
mu_SGy128_4_lights = np.array([0, 0, 0.01, 0.01, 0.1, 0.1, 1, 1, 0.03, 0.03, 0.01, 0.01, 0.06, 0.06, 0.08, 0.08])


fig = plt.figure(figsize=(10,5))
ax = fig.add_subplot(121)
ax.plot(u_arr, mu_arr_invert/max(mu_arr_invert), '-x', label='mu')
ax.plot(mu_SGy104_1_lights, mu_SGy104_1/0.45, 'ob', label='mu')
ax.set_xscale('symlog', linthresh=0.01)

ax = fig.add_subplot(122)
ax.plot(u_arr, mu_arr_amp/max(mu_arr_amp), '-x', label='mu')
ax.plot(mu_SGy128_4_lights, mu_SGy128_4/0.45, 'ob', label='mu')
ax.set_xscale('symlog', linthresh=0.01)

fig = plt.figure(figsize=(10,5))
ax = fig.add_subplot(121)
ax.plot(u_arr, steady_state_arr_invert[:,2], '-', color='red', label='mu')
ax.plot(u_arr, steady_state_arr_invert[:,4], '-', color='blue', label='mu')
ax.set_xscale('symlog', linthresh=0.01)

ax = fig.add_subplot(122)
ax.plot(u_arr, steady_state_arr_amp[:,3], '-', color='red', label='mu')
ax.plot(u_arr, steady_state_arr_amp[:,5], '-', color='blue', label='mu')
ax.set_xscale('symlog', linthresh=0.01)



NameError: name 'Ks' is not defined

In [ ]:
mu_SGy128_4_rejects = np.array([0.26858448, 0.28688688, 0.19000113,  0.25622547, 0.26932193, 0.28158447, 0.26857221])
mu_SGy128_4_rejects_lights = np.array([0,  0.01, 0.01, 1, 1,  0.01, 0.01])

mu_SGy128_4_trues = np.array([0.07654071, 0.3834206 ,
       0.42503585, 0.30298274, 0.29615459,
       0.34449419, 0.38737338, 0.412247  ,
       0.43290314])
mu_SGy128_4__trues_lights = np.array([0, 0.1, 0.1, 0.03, 0.03, 0.06, 0.06, 0.08, 0.08])


fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111)
ax.plot(u_arr, mu_arr_invert, '-b', label='mu', linewidth=3)
ax.plot(mu_SGy104_1_lights, mu_SGy104_1, 'ob', label='mu')
ax.set_xscale('symlog', linthresh=0.01)
ax.tick_params(axis='both', which='major', labelsize=20)
ax.set_xlabel('Light Pulsing Fraction', fontsize=30)
ax.set_ylabel(r'Growth rate $\mu$ (1/h)', fontsize=30)
ax.set_title('Inverted Circuit Dose Response', fontsize=30)
ax.set_ylim([0, 0.5])
[i.set_linewidth(3) for i in ax.spines.values()]

fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111)
ax.plot(u_arr, mu_arr_amp, '-g', label='mu', linewidth=3)
ax.plot(mu_SGy128_4__trues_lights, mu_SGy128_4_trues, 'og', label='mu')
ax.plot(mu_SGy128_4_rejects_lights, mu_SGy128_4_rejects, 'xr', label='mu')
ax.set_xscale('symlog', linthresh=0.01)
ax.tick_params(axis='both', which='major', labelsize=20)
ax.set_xlabel('Light Pulsing Fraction', fontsize=30)
ax.set_ylabel(r'Growth rate $\mu$ (1/h)', fontsize=30)
ax.set_title('Amp Circuit Dose Response', fontsize=30)
ax.set_ylim([0, 0.5])
[i.set_linewidth(3) for i in ax.spines.values()]

fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111)
ax.plot(u_arr, mu_arr_amp, '-g', label='Amp Circuit', linewidth=3)
# ax.plot(mu_SGy128_4__trues_lights, mu_SGy128_4_trues, 'og')
ax.plot(u_arr, mu_arr_invert, '-b', label='Invert Circuit', linewidth=3)
# ax.plot(mu_SGy104_1_lights, mu_SGy104_1, 'ob')
# ax.plot(mu_SGy128_4_rejects_lights, mu_SGy128_4_rejects, 'xr', label='mu')
ax.set_xscale('symlog', linthresh=0.01)
ax.tick_params(axis='both', which='major', labelsize=20)
ax.set_xlabel('Light Pulsing Fraction', fontsize=30)
ax.set_ylabel(r'Growth rate $\mu$ (1/h)', fontsize=30)
ax.set_title('Dose Response Curves', fontsize=30)
plt.legend(fontsize=20)
ax.set_ylim([0, 0.5])
[i.set_linewidth(3) for i in ax.spines.values()]

In [ ]:
from scipy.optimize import fsolve
from scipy.interpolate import interp1d

macro_period = 2
macro_pulse = 0.9
t_final = 48

od_sp = 0.01

S0 = 20

optogrowth_p = kdegR, ktrans, kdegP, kdegPPDC, mu_max_invert, mu_max_amp, Ks, Y, K_Rep, K_Amp, n_Rep, n_Amp

mRNA_invert_fun = lambda  u: interp1d(u_arr, mRNA_arr_invert)(u)
mRNA_amp_fun = lambda  u: interp1d(u_arr, mRNA_arr_amp)(u)

def macropulse_fsolvefun(x, optogrowth_p, plot=False):

    global setpoint_invert, setpoint_amp

    setpoint_invert = np.array([0, mRNA_invert_fun(1)])
    setpoint_amp = np.array([0, mRNA_amp_fun(1)])

    # print(setpoint_invert)
    # print(setpoint_amp)
    # print('--------------------')

    x[-3] = S0 - od_sp / Y
    x[-2] = od_sp
    x[-1] = 0

    sol = solve_ivp(opto_growth, t_span=np.array([0, macro_pulse*macro_period*60]), y0=x, args=(optogrowth_p,od_sp),
                        rtol=1e-6, atol=1e-9, 
                        method='BDF',
                        first_step = 0.0001,
                        )
#     print('solveIvp', sol.message)
    x_init_inv = sol.y[:,-1]

    setpoint_invert = np.array([0, mRNA_invert_fun(0)])
    setpoint_amp = np.array([0, mRNA_amp_fun(0)])
    # print(setpoint_invert)
    # print(setpoint_amp)
    # print('===============================')

    sol2 = solve_ivp(opto_growth, t_span=np.array([macro_pulse*macro_period*60, macro_period*60]), y0=x_init_inv, args=(optogrowth_p,od_sp),
                        rtol=1e-6, atol=1e-9, 
                        method='BDF',
                        first_step = 0.0001,
                        )
#     print('solveIvp', sol.message)
    x_init_inv = sol2.y[:,-1]

    if plot:
        fig = plt.figure(figsize=(10,5))
        ax = fig.add_subplot(111)
        ax.plot(sol.t/60, sol.y[0], '-b', label='OD')
        ax.plot(sol2.t/60, sol2.y[0], '-r', label='OD2')

    return (x_init_inv - x)

optogrowth_p = kdegR, ktrans, kdegP, kdegPPDC, mu_max_invert, 0, Ks, Y, K_Rep, K_Amp, n_Rep, n_Amp
x_guess = [1, 1, 1, 1, 1, 1, 20, 0.1, 0.0]
x_init_invert, infodict, ier, mesg = fsolve(macropulse_fsolvefun, x_guess, args=(optogrowth_p,), xtol=1e-9, maxfev=10000, full_output=True, factor=0.0001)
# print('invert', ier, mesg)
# print(x_init_invert)

macropulse_fsolvefun(x_init_invert, optogrowth_p, plot=True)

# mRNA_invert, mRNA_amp, Rep, Amp, Enz_invert, Enz_amp, S, X_invert, X_amp = x

In [ ]:
from scipy.optimize import fsolve
from scipy.interpolate import interp1d

macro_period = 1
macro_pulse_arr = np.flip(10 ** np.linspace(-3,-0.01,20))
t_final = 48

od_sp = 0.01

S0 = 20

optogrowth_p = kdegR, ktrans, kdegP, kdegPPDC, mu_max_invert, mu_max_amp, Ks, Y, K_Rep, K_Amp, n_Rep, n_Amp

mRNA_invert_fun = lambda  u: interp1d(u_arr, mRNA_arr_invert)(u)
mRNA_amp_fun = lambda  u: interp1d(u_arr, mRNA_arr_amp)(u)

ss_Enz_invert_arr = []
ss_Enz_amp_arr = []

for i, macro_pulse in tqdm(enumerate(macro_pulse_arr), total=len(macro_pulse_arr)):
    # print(macro_pulse)

    def macropulse_fsolvefun(x, optogrowth_p, initvals, plot=False):

        global setpoint_invert, setpoint_amp

        setpoint_invert = np.array([0, mRNA_invert_fun(1)])
        setpoint_amp = np.array([0, mRNA_amp_fun(1)])

        # print(setpoint_invert)
        # print(setpoint_amp)
        # print('--------------------')
        x[-3:] = initvals

        # x[-3] = S0 - od_sp / Y
        # x[-2] = od_sp
        # x[-1] = 0

        sol = solve_ivp(opto_growth, t_span=np.array([0, macro_pulse*macro_period*60]), y0=x, args=(optogrowth_p,od_sp),
                            rtol=1e-6, atol=1e-9, 
                            method='BDF',
                            first_step = 0.0001,
                            )
    #     print('solveIvp', sol.message)
        x_init_inv = sol.y[:,-1]

        setpoint_invert = np.array([0, mRNA_invert_fun(0)])
        setpoint_amp = np.array([0, mRNA_amp_fun(0)])
        # print(setpoint_invert)
        # print(setpoint_amp)
        # print('===============================')

        sol2 = solve_ivp(opto_growth, t_span=np.array([macro_pulse*macro_period*60, macro_period*60]), y0=x_init_inv, args=(optogrowth_p,od_sp),
                            rtol=1e-6, atol=1e-9, 
                            method='BDF',
                            first_step = 0.0001,
                            )
    #     print('solveIvp', sol.message)
        x_init_inv = sol2.y[:,-1]

        if plot:
            fig = plt.figure(figsize=(10,5))
            ax = fig.add_subplot(111)
            ax.plot(sol.t/60, sol.y[4], '-b', label='OD')
            ax.plot(sol2.t/60, sol2.y[4], '-r', label='OD2')

        return (x_init_inv - x)

    def macropulse_ssfun(x, optogrowth_p, initvals, plot=False):

        global setpoint_invert, setpoint_amp

        setpoint_invert = np.array([0, mRNA_invert_fun(1)])
        setpoint_amp = np.array([0, mRNA_amp_fun(1)])

        # print(setpoint_invert)
        # print(setpoint_amp)
        # print('--------------------')

        x[-3:] = initvals

        # x[-3] = S0 - od_sp / Y
        # x[-2] = od_sp
        # x[-1] = 0

        sol = solve_ivp(opto_growth, t_span=np.array([0, macro_pulse*macro_period*60]), y0=x, args=(optogrowth_p,od_sp),
                            rtol=1e-6, atol=1e-9, 
                            method='BDF',
                            first_step = 0.0001,
                            t_eval=np.linspace(0, macro_pulse*macro_period*60, 1000)
                            )
    #     print('solveIvp', sol.message)
        x_init_inv = sol.y[:,-1]

        setpoint_invert = np.array([0, mRNA_invert_fun(0)])
        setpoint_amp = np.array([0, mRNA_amp_fun(0)])
        # print(setpoint_invert)
        # print(setpoint_amp)
        # print('===============================')

        sol2 = solve_ivp(opto_growth, t_span=np.array([macro_pulse*macro_period*60, macro_period*60]), y0=x_init_inv, args=(optogrowth_p,od_sp),
                            rtol=1e-6, atol=1e-9, 
                            method='BDF',
                            first_step = 0.0001,
                            t_eval=np.linspace(macro_pulse*macro_period*60, macro_period*60, 1000)
                            )
    #     print('solveIvp', sol.message)
        x_init_inv = sol2.y[:,-1]

        ss_arr = (np.trapz(sol2.y, sol2.t, axis=1) + np.trapz(sol.y, sol.t, axis=1)) / (macro_period*60)

        return ss_arr

    optogrowth_p = kdegR, ktrans, kdegP, kdegPPDC, mu_max_invert, 0, Ks, Y, K_Rep, K_Amp, n_Rep, n_Amp
    x_guess = [1, 1, 1, 1, 1, 1, 20, 0.1, 0.0]
    initvals =  [S0 - od_sp / Y, od_sp, 0]
    x_init_invert, infodict, ier, mesg = fsolve(macropulse_fsolvefun, x_guess, args=(optogrowth_p,initvals), xtol=1e-9, maxfev=10000, full_output=True, factor=0.0001)
    x_ss = macropulse_ssfun(x_init_invert, optogrowth_p, initvals)
    ss_Enz_invert_arr.append(x_ss[4])
    # macropulse_fsolvefun(x_init_invert, optogrowth_p, initvals, plot=True)


    optogrowth_p = kdegR, ktrans, kdegP, kdegPPDC, 0, mu_max_amp, Ks, Y, K_Rep, K_Amp, n_Rep, n_Amp
    x_guess = [1, 1, 1, 1, 1, 1, 20, 0.0, 0.1]
    initvals =  [S0 - od_sp / Y, 0, od_sp]
    x_init_amp, infodict, ier, mesg = fsolve(macropulse_fsolvefun, x_guess, args=(optogrowth_p,initvals), xtol=1e-9, maxfev=10000, full_output=True, factor=0.0001)
    x_ss = macropulse_ssfun(x_init_amp, optogrowth_p,   initvals)
    ss_Enz_amp_arr.append(x_ss[5])
    # macropulse_fsolvefun(x_init_amp, optogrowth_p, initvals, plot=True)

ss_Enz_invert_arr = np.array(ss_Enz_invert_arr)
ss_Enz_amp_arr = np.array(ss_Enz_amp_arr)

mu_arr_invert_macro = mu_max_invert * ss_Enz_invert_arr * S0 / (Ks + S0) * 60
mu_arr_amp_macro = mu_max_amp * ss_Enz_amp_arr * S0 / (Ks + S0) * 60

In [ ]:
# mu_arr_invert = mu_max_invert * ss_Enz_invert_arr * S0 / (Ks + S0) * 60
# mu_arr_amp = mu_max_amp * ss_Enz_amp_arr * S0 / (Ks + S0) * 60

## Real Data
mu_SGy104_1 = np.array([0.429016493, 0.419454997, 0.425278485, 0.468815929, 0.214066814, 0.202848859, 0.077447387, 0.067285511, 0.487083379, 0.487241995,
0.436233624, 0.416734105, 0.433773399, 0.4552301, 0.406627518, 0.322757789])
mu_SGy104_1_lights = np.array([0, 0, 0.01, 0.01, 0.1, 0.1, 1, 1, 0.03, 0.03, 0.01, 0.01, 0.06, 0.06, 0.08, 0.08])

mu_SGy128_4 = np.array([0.07654071, 0.26858448, 0.28688688, 0.19000113, 0.3834206 ,
       0.42503585, 0.25622547, 0.26932193, 0.30298274, 0.29615459,
       0.28158447, 0.26857221, 0.34449419, 0.38737338, 0.412247  ,
       0.43290314])
mu_SGy128_4_lags = np.array([ 0.        , 39.88279932, 19.98011315, 14.4137813 ,  0.        ,
        2.18260617,  0.        ,  1.21280087, 10.40812555,  7.77763666,
       21.22069601, 19.70771971,  1.68719277,  3.11811039,  2.52691981,
        2.30179095])
mu_SGy128_4_lights = np.array([0, 0, 0.01, 0.01, 0.1, 0.1, 1, 1, 0.03, 0.03, 0.01, 0.01, 0.06, 0.06, 0.08, 0.08])


fig = plt.figure(figsize=(15,5))
ax = fig.add_subplot(131)
ax.plot(macro_pulse_arr, mu_arr_invert_macro, '-x', label='mu')
ax.plot(mu_SGy104_1_lights, mu_SGy104_1, 'ob', label='mu')
ax.set_xscale('symlog', linthresh=0.01)

ax = fig.add_subplot(132)
ax.plot(macro_pulse_arr, mu_arr_amp_macro, '-x', label='mu')
ax.plot(mu_SGy128_4_lights, mu_SGy128_4, 'ob', label='mu')
ax.set_xscale('symlog', linthresh=0.01)

ax = fig.add_subplot(133)
ax.plot(macro_pulse_arr, mu_arr_amp_macro, '--g', label='mu')
ax.plot(macro_pulse_arr, mu_arr_invert_macro, '--b', label='mu')
ax.plot(u_arr, mu_arr_amp, '-g', label='Amp Circuit', linewidth=3)
# ax.plot(mu_SGy128_4__trues_lights, mu_SGy128_4_trues, 'og')
ax.plot(u_arr, mu_arr_invert, '-b', label='Invert Circuit', linewidth=3)
ax.set_xscale('symlog', linthresh=0.01)
